In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint



paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
Using TensorFlow backend.


In [17]:
test = pd.read_csv("EN_PT.csv",usecols=['EN'])

In [18]:
test.head()

,EN
0,i have lost my passport
1,someone stole my money
2,help
3,may i have the bill
4,i would like dessert


In [19]:
test = test.assign(Sentence = "we are best friends")

In [20]:
test.head()

,EN,Sentence
0,i have lost my passport,we are best friends
1,someone stole my money,we are best friends
2,help,we are best friends
3,may i have the bill,we are best friends
4,i would like dessert,we are best friends


In [9]:
EMBEDDING_FILE = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'

In [6]:
stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# Prepare embedding
def make_w2v_embeddings(df, embedding_dim=300, empty_w2v=False):
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
    word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

    questions_cols = ['EN', 'Sentence']

    # Iterate over the questions only of both training and test datasets
    for dataset in [df]:
        for index, row in dataset.iterrows():

            # Iterate through the text of both questions of the row
            for question in questions_cols:

                sent = []  # q2n -> question numbers representation
                for word in text_to_word_list(row[question]):

                    # Check for unwanted words
                    if word in stops and word not in word2vec.vocab:
                        continue

                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        sent.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        sent.append(vocabulary[word])

                # Replace questions as word to question as number representation
                dataset.set_value(index, question, sent)

    embedding_dim = 300
    embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0  # So that the padding will be ignored

    # Build the embedding matrix
    for word, index in vocabulary.items():
        if word in word2vec.vocab:
            embeddings[index] = word2vec.word_vec(word)

    #del word2vec
    #word2vec.save_word2vec_format('EMBEDDING_FILE', binary=False)
    return df, embeddings, word2vec

In [7]:
def split_and_zero_padding(df, max_seq_length):
    # Split to dicts
    X = {'left': df['EN'], 'right': df['Sentence']}

    # Zero padding
    for dataset, side in itertools.product([X], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

    return dataset

In [21]:
# Make word2vec embeddings
embedding_dim = 300
max_seq_length = 212

test_set = test.copy()

test_df, embeddings, word2vec = make_w2v_embeddings(test, embedding_dim=embedding_dim, empty_w2v=False)

# Split to dicts and append zero padding.
X_test = split_and_zero_padding(test_df, max_seq_length)

# Make sure everything is ok
assert X_test['left'].shape == X_test['right'].shape

/usr/local/Cellar/ipython/6.3.1/libexec/vendor/lib/python3.6/site-packages/ipykernel_launcher.py:73: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [12]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [22]:
from keras.models import load_model

model = load_model('malstm_trained_en.h5', custom_objects={'exponent_neg_manhattan_distance': exponent_neg_manhattan_distance})

In [23]:
prediction = model.predict([X_test['left'], X_test['right']])
idx = np.argmax(prediction)
print(idx)

179


In [24]:
test_set.iloc[idx]

EN          we are good friends
Sentence    we are best friends
Name: 179, dtype: object